In [1]:
import os
from pathlib import Path

import numpy as np
import scanpy as sc
try:
    import hnswlib
    hnswlib_imported = True
except ImportError:
    hnswlib_imported = False
    print("hnswlib not installed! We highly recommend installing it for fast similarity search.")
    print("To install it, run: pip install hnswlib")

try:
    n_available_cores = len(os.sched_getaffinity(0))
    print("Number of available cores: {}".format(n_available_cores))
except Exception:
    n_available_cores = min(10, os.cpu_count())

Number of available cores: 6


In [2]:
# loop over all the embedding datasets to collect the embeddings

In [3]:
# REFERENCE
ref_data_dir = "/scratch/ssd004/datasets/cellxgene/embed/brain/"
# get all file under this dir
ref_data_path_list = os.listdir(ref_data_dir)
ref_data_path_list = [os.path.join(ref_data_dir, i) for i in ref_data_path_list]

In [4]:
# QUERY
query_data_dir = "/scratch/ssd004/datasets/cellxgene/embed_dataset/ms-dataset/"
# get all file under this dir
query_data_path_list = os.listdir(query_data_dir)
# query_data_path_list = [os.path.join(query_data_dir, i) for i in query_data_path_list]
query_data_path_list = [
    os.path.join(query_data_dir, i) for i in query_data_path_list if "embed_0" in i
]
query_data_path_list

['/scratch/ssd004/datasets/cellxgene/embed_dataset/ms-dataset/embed_0.h5ad']

In [5]:
# read all data
embed_list = []
meta_list = []
for ref_data_path in ref_data_path_list:
    data = sc.read_h5ad(ref_data_path)
    embed_list.append(data.X)
    meta_list.append(data.obs["cell_type"].values)

embed_array = np.concatenate(embed_list, axis=0)
meta_array = np.concatenate(meta_list, axis=0)
del embed_list, meta_list

In [6]:
data

AnnData object with n_obs × n_vars = 200000 × 512
    obs: 'soma_joinid', 'cell_type', 'assay', 'tissue', 'tissue_general'

In [7]:
embed_array.shape

(13225889, 512)

In [8]:
# Declaring index, using most of the default parameters from https://github.com/nmslib/hnswlib
p = hnswlib.Index(space = 'l2', dim = embed_array.shape[1]) # possible options are l2, cosine or ip
p.init_index(max_elements = embed_array.shape[0], ef_construction = 30, M = 16)

# Element insertion (can be called several times):
p.add_items(embed_array)

# Controlling the recall by setting ef:
# p.set_ef(15) # ef should always be > k

p.ef


10

## ways to reduce the memory footprint of the index

1. use smaller feature dimensions. Create an autoencoder or PCA before indexing? An issue with the projection will need to be updated now and then when a large trunk of new data is added to the atlas.
    
    1.1. Also, if using PCA, be aware that the different dimensions can have different variance scales. So it is important to use a distance metric such as l2 instead of cosine.
    
2. use smaller number of cells, such as meta cells. A naive way to select meta cells is to do an stratified sampling of the cells of different cell types in the atlas. The strategy can be tricky and will need to be updated when new data is added as well.

    2.1 On one hand, using some meta cells may be needed. Since there can be too many cells, we want the found K neighbors to be representative enough, like they can cover enough range in the cell space. To make this more clear, some area in the cell space may be too dense comparing to other areas, this will make it an unfair higher chance of selecting cells in the region. So basically too approaches to relieve this (1) selecting meta cells more evenly spread, and then just retrieve in some meta cell space. (2) Post processing like normalized voting across cell types. A summarizing point is as follows, even if you want to use the approach (2), one will still need approach (1) because you will at least need cell states that should be covered actually get covered, so that then they can be re weighted during the voting process.

    2.2 Using stratified sampling by annotated cell types is not appropriate, since the annotation schema can definitely change over time. So really need to make sure the meta cell selection is simply dependent on the raw data themselves.

Another point is that when computing the retrieving process. Balancing the voting process by considering the different numbers of cells for each cell is essential. Maybe the voting weights can be normalized by the ratio of cells in the atlas. Also please have a look at existing literature about how they applied that.

In [9]:
index_save_path = "./index.bin"
print(f"Saving index to {index_save_path}")
p.save_index(index_save_path)

Saving index to ./index.bin


In [10]:
query_embed_list = []
query_meta_list = []
for query_data_path in query_data_path_list:
    data = sc.read_h5ad(query_data_path)
    query_embed_list.append(data.X)
    query_meta_list.append(data.obs["cell_type"].values)
    
query_embed_array = np.concatenate(query_embed_list, axis=0)
query_meta_array = np.concatenate(query_meta_list, axis=0)


In [11]:
k = 10
# Query dataset, k - number of closest elements (returns 2 numpy arrays)
# idx, distances = p.knn_query(query_embed_array, k = k)
idx, distances = p.knn_query(query_embed_array[:100], k = k)
gt = query_meta_array[:100]

In [12]:
matched_array = meta_array[idx]
from scipy.stats import mode

voting = mode(matched_array, axis=1)[0]

### TODO: need a weighted voting to balance the different ratio of celltypes in the reference atlas

In [13]:
voting[:10]

array([['oligodendrocyte'],
       ['cerebral cortex GABAergic interneuron'],
       ['oligodendrocyte'],
       ['oligodendrocyte'],
       ['oligodendrocyte'],
       ['cerebral cortex GABAergic interneuron'],
       ['astrocyte'],
       ['neuron'],
       ['astrocyte'],
       ['oligodendrocyte']], dtype=object)

In [14]:
gt[:10]

array(['oligodendrocyte A', 'PVALB-expressing interneuron',
       'oligodendrocyte A', 'oligodendrocyte precursor cell',
       'oligodendrocyte A', 'mixed glial cell?', 'mixed glial cell?',
       'VIP-expressing interneuron', 'astrocyte',
       'oligodendrocyte precursor cell'], dtype=object)